In [13]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
from Tkinter import Tk
from tkFileDialog import askopenfilename
import os
from os import path
from IPython.display import HTML,display

In [1]:
def get_avg_write(scenarios,event,databases):
    databases = databases
    herzs = ["30hz","30hz","60hz","60hz","120hz","120hz"]
    column_labels = ["write(30hz)","write_with_read(30hz)",
                     "write(60hz)","write_with_read(60hz)",
                     "write(120hz)","write_with_read(120hz)"]
    avgs = np.zeros((0,len(databases)))

    _flag = True
    
    for herz in herzs:
        avg = np.zeros((1,0))
        for database in databases:
            directory = get_directory_path(database,herz,scenarios[0] if _flag else scenarios[1])
            file = directory+get_log_file_name(directory)
            with open(file) as fp:
                lines = fp.readlines()
            times = []
            for line in lines:
                if database is "neo4j" and "Robot_id" in line and event[0] in line:
                    times.append(float(line.split(" ")[7]))
                elif "Robot_id" in line and event[1] in line:
                    times.append(float(line.split(" ")[8]))
            times = outlier_removal(times)
            avg = np.hstack([avg,[[round(np.mean(times),4)]]])
        _flag = False if (_flag is True) else True
        avgs = np.vstack([avgs,avg])
    mongodb_df_write_1 = pd.DataFrame(avgs, columns=databases,
                   index=column_labels)
    return mongodb_df_write_1

def get_avg_read(scenarios,query,databases):
    databases = databases
    herzs = ["30hz","30hz","60hz","60hz","120hz","120hz"]
    column_labels = ["read(30hz)","read_with_write(30hz)",
                     "read(60hz)","read_with_write(60hz)",
                     "read(120hz)","read_with_write(120hz)"]
    
    avgs = np.zeros((0,len(databases)))
    
    _flag = True
    
    for herz in herzs:
        avg = np.zeros((1,0))
        for database in databases:
            directory = get_directory_path(database,herz,scenarios[0] if _flag else scenarios[1])
            file = directory+get_log_file_name(directory)
            with open(file) as fp:
                lines = fp.readlines()
            times = []
            for line in lines:
                if "Robot_id" in line and query in line:
                    times.append(float(line.split(" ")[8]))
            times = outlier_removal(times)
            avg = np.hstack([avg,[[round(np.mean(times),4)]]])
        _flag = not _flag
        avgs = np.vstack([avgs,avg])
    df = pd.DataFrame(avgs, columns=databases,
                   index=column_labels)
    return df

def get_directory_path(db_name,herz,scenario):
    return "./logs/"+db_name+"/"+scenario+"/"+herz+"/"

def get_log_file_name(directory):
    return [f for f in os.listdir(directory)][0]

def plot(df,title):
    fig, axes = plt.subplots()
    fig.set_figheight(8)
    fig.set_figwidth(17)

    plot = df.plot(ax=axes,kind='bar',table=True,grid=True,title=title)
    
    axes.set_xticklabels([])
#     plot.set_xlabel("Frequency of generating events (Herz)")
    plot.set_ylabel("Average query execution time (Seconds)")
    
     
def outlier_removal(times):
    result = []
    
    mean = np.mean(times)
    std = np.std(times)
    
    for time in times:
        if abs(time-mean) < 2*std:
            result.append(time)
    return result

In [86]:
# def outlier_removal(file_path,event):
# #     mu  = mean of the data
# # std = standard deviation of the data
# # IF abs(x-mu) > 3*std  THEN  x is outlier
#     with open(file_path) as fp:
#         lines = fp.readlines()
    
# #     times = np.zeros((1,0))
#     times = []
#     for line in lines:
#         if "Robot_id" in line and event in line:
# #             times = np.hstack([times,[[float(line.split(" ")[8])]]])
#             times.append(float(line.split(" ")[8]))
#     mean = np.mean(times)
#     std = np.std(times)
#     print mean,std
    
#     counter = 0
#     for time in times:
#         if abs(time-mean) > 2*std:
#             print time
#             counter +=1
#     print counter

# directory = get_directory_path("arangodb","60hz","rw_read")
# file_path = directory+get_log_file_name(directory)
# outlier_removal(file_path,"Query_id : 0")

113.828679482 47.136524453
11.1280760765
1
